In [ ]:
type Node{T}
    left::Nullable{Node{T}}
    right::Nullable{Node{T}}
    value::T
end

Node{T}(v::T) = Node(Nullable{Node{T}}(), Nullable{Node{T}}(), v)

In [1]:
n1 = Node(Nullable(Node(2)), Nullable(Node(3)), 10)

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [12]:
left(n::Node) = isnull(n.left) ? Void(): get(n.left)
right(n::Node) = isnull(n.right) ? Void(): get(n.right)

right (generic function with 1 method)

In [ ]:
function search{T}(graph::Node{T}, number)
    for i = 1:10
        if graph.value == number
            return number
        else
            try
                graph = graph.left.value
            catch
                return 0
            end
        end
    end
end
    

In [1]:
include("SymmetricMatrix.jl")
include("Test.jl")
using Test

In [3]:
include("SymmetricMatrix.jl")
using SymmetricMatrix
using NullableArrays
importall SymmetricMatrix

In [1]:
using Iterators
function genstensor(T::Type, dims::Int, l::Int, seed::Int = 1234)
    srand(seed)
    ret = zeros(T, fill(l, dims)...)
    indices = collect(product(fill(collect(1:l), dims)...))
    elements = Array{T,1}(randn(l^dims÷dims))
    r = 1
    for i in indices 
        if issorted(i)          
            for k in collect(permutations(i))
                ret[k...] = elements[r]
            end
            r += 1
        end
    end
    ret
end

genstensor (generic function with 2 methods)

In [4]:
l = 75
st = genstensor(Float64, 4,l);
st1 = genstensor(Float64, 4,l, 1233);
c = randn(l,12);

In [5]:
bs = (convert(BoxStructure{Float64}, st, 10)) ;
@time bs1 = (convert(BoxStructure{Float64}, st1, 10)) ;

  0.368854 seconds (320.15 k allocations: 589.424 MB, 32.72% gc time)


In [6]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;

  0.107971 seconds (259.60 k allocations: 13.942 MB)
  0.080173 seconds (322.12 k allocations: 17.625 MB, 4.54% gc time)
  0.013778 seconds (7 allocations: 47.684 MB, 19.70% gc time)
  0.012109 seconds (7 allocations: 47.684 MB, 10.60% gc time)


In [7]:
@time convert(Array{Float64}, bs);
@time vec(bs);
@time vec(st);

  0.338190 seconds (1.23 M allocations: 148.155 MB, 15.07% gc time)
  0.297563 seconds (1.21 M allocations: 147.300 MB, 15.78% gc time)
  0.000001 seconds (6 allocations: 240 bytes)


In [8]:
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;

  0.046923 seconds (217.14 k allocations: 12.077 MB, 5.30% gc time)
  0.041683 seconds (211.89 k allocations: 11.764 MB)
  0.046537 seconds (217.14 k allocations: 12.077 MB, 5.47% gc time)
  0.018239 seconds (13 allocations: 47.684 MB, 7.72% gc time)
  0.015898 seconds (7 allocations: 47.684 MB, 4.35% gc time)
  0.049982 seconds (13 allocations: 47.684 MB, 1.66% gc time)


In [8]:
@time modemult(bs, c', 4)
@time Tensors.modemult(st,c',4);

  3.900043 seconds (5.54 M allocations: 6.035 GB, 6.03% gc time)
  1.293228 seconds (176 allocations: 946.063 MB, 2.76% gc time)


In [13]:
using Tensors
using NullableArrays
using Iterators
using Base.size

T1 = randn(5,5);
r = randn(13,13)
r1 = randn(13,5)
c = covbs(r, 3);
c1 = cov(r, corrected = false);


seg(i::Int, of::Int, limit::Int) = (i*of <= limit)? (((i-1)*of+1):(i*of)): (((i-1)*of+1):limit)


readsegments{T <: AbstractFloat}(i::Array{Int}, bs::BoxStructure{T}) = permutedims(bs.frame[sort(i)...].value, invperm(sortperm(i)))


segmentmult{T <: AbstractFloat, N}(k::Array{Int, 1}, bsdata::BoxStructure{T}, m::NullableArray{Array{T, N}}) =
mapreduce(i -> readsegments([k[1],i], bsdata)*(m[i,k[2]].value), +, 1:size(bsdata)[2])

segmentmult{T <: AbstractFloat, N}(k::Array{Int, 1}, m::NullableArray{Array{T, N}}, m1::NullableArray{Array{T, N}}) =
mapreduce(i -> (m[i, k[1]].value)'*(m1[i,k[2]].value), +, 1:Base.size(m1, 1))

function slisemat{T <: AbstractFloat}(mat::Matrix{T}, slisesize::Int)
    segments = ceil(Int, [Base.size(mat)...]/slisesize)
    ret = NullableArray(Array{T, 2}, segments...)
    for k in product(1:segments[1], 1:segments[2])
        ret[k...] = mat[seg(k[1], slisesize, Base.size(mat, 1)),seg(k[2], slisesize, Base.size(mat, 2))]
    end
    ret
end

function bcss{T <: AbstractFloat, N}(bsdata::BoxStructure{T, N}, m::Matrix{T})
    s = size(bsdata)
    Base.size(m,1) == s[3] || throw(DimensionMismatch("..."))
    m = slisemat(m, s[1])
    ret = similar(m)
    for i = 1:s[2], j = 1:Base.size(m,2)
        ret[i,j] = segmentmult([i,j], bsdata, m)
    end
    ret1 = NullableArray(Array{T, 2}, Base.size(m,2), Base.size(m,2))
    for j = 1:Base.size(m,2), i = 1:j
        ret1[i,j] = segmentmult([i,j], m, ret)
    end
    BoxStructure(ret1)
end

bcss (generic function with 1 method)

In [14]:
convert(Array{Float64}, bcss(c, r1)) - r1'*(c*r1)

5x5 Array{Float64,2}:
  0.0           4.44089e-16  -8.88178e-16   0.0           0.0        
  1.11022e-16   0.0           0.0           0.0           0.0        
 -4.44089e-16  -1.77636e-15  -8.88178e-16   0.0           4.44089e-16
  0.0           0.0           4.44089e-16   3.55271e-15  -8.88178e-16
  8.88178e-16   4.44089e-16  -1.11022e-15  -8.88178e-16   8.88178e-16

In [118]:
slisemat(r1, 3)

4x3 NullableArrays.NullableArray{Array{Float64,2},2}:
 3x3 Array{Float64,2}:
 -0.353055  -0.506382   -2.50666 
 -0.924935   1.97074    -0.258612
  0.394801  -0.0753163   0.994687  …  3x2 Array{Float64,2}:
  0.0174044  -0.217459 
 -0.589807   -0.253531 
 -0.509182    0.0879423
 3x3 Array{Float64,2}:
  1.75426   1.60086  -0.56398 
 -0.264379  1.27616  -0.323865
 -1.27033   1.04024   0.645991              3x2 Array{Float64,2}:
 -0.752288  -0.0916412
  0.680844  -0.131055 
  1.10156   -0.596527    
 3x3 Array{Float64,2}:
  0.718838   2.45473  -0.9615  
 -0.172207  -1.68454  -0.678976
 -1.88238    0.19797  -1.05534            3x2 Array{Float64,2}:
 0.986525  -0.447888
 0.516853   0.730334
 0.562838   0.246931         
 3x3 Array{Float64,2}:
  0.728548  -1.13779   -0.641451 
 -0.453574   0.664553  -0.0321457
 -0.380402  -1.75362    1.29153       3x2 Array{Float64,2}:
 -0.138364  -0.0835277
  1.59715   -0.039286 
  1.66072   -0.361203    

In [8]:
a + a

LoadError: LoadError: 
while loading In[8], in expression starting on line 1

In [9]:
for f in (
    :+,
    :-,
)
    @eval begin
        @inline function $(f){S1, S2}(x::Nullable{Matrix{S1}}, y::Nullable{Matrix{S2}}, i::Int)
            Nullable($(f)(x.value, y.value), x.isnull | y.isnull)
        end
    end
end


In [4]:
r = randn(7,7)
a = covbs(r);


In [6]:
a*0

SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [8]:
*(a, 0, true)


SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [9]:
a

SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [3]:
+!(2,2)

LoadError: LoadError: MethodError: `!` has no method matching !(::Int64, ::Int64)
while loading In[3], in expression starting on line 1